Introduction
At its core, churn prediction is a classification problem, where the classes often are ‘churned’ and ‘active’. The prediction is based on historical data, including customer behavior, demographics, transaction history, and more.

Goal: well-balanced classification model.

Instructions:
Work in Python or R.
Examine the data.
While working through the layers of challenge, please leave comments in your code.
Share your solution in a notebook format with us.



In [15]:
import pandas as pd
import numpy as np

# Loading and displaying dataset

In [2]:
churn_df = pd.read_csv("../data/task_data_churned.csv")

In [3]:
pd.set_option('display.max_columns', None)

# Understanding columns


In [4]:
churn_df.columns

Index(['ws_users_activated', 'ws_users_deactivated', 'ws_users_invited',
       'action_create_project', 'action_export_report',
       'action_api_and_webhooks', 'action_time_entries_via_tracker',
       'action_start_trial', 'action_import_csv', 'action_create_invoice',
       'action_lock_entries', 'action_add_targets',
       'action_connect_quickbooks', 'action_create_expense',
       'action_project_budget', 'action_gps_tracking', 'action_screenshots',
       'action_create_custom_field', 'country', 'value_days_to_purchase',
       'value_number_of_active_months', 'value_transactions_number',
       'value_regular_seats', 'value_kiosk_seats', 'revenue',
       'churned_status'],
      dtype='object')

In [5]:
churn_df.dtypes

ws_users_activated                   int64
ws_users_deactivated                 int64
ws_users_invited                     int64
action_create_project                int64
action_export_report                 int64
action_api_and_webhooks              int64
action_time_entries_via_tracker      int64
action_start_trial                   int64
action_import_csv                    int64
action_create_invoice                int64
action_lock_entries                  int64
action_add_targets                   int64
action_connect_quickbooks            int64
action_create_expense                int64
action_project_budget                int64
action_gps_tracking                float64
action_screenshots                 float64
action_create_custom_field         float64
country                             object
value_days_to_purchase               int64
value_number_of_active_months        int64
value_transactions_number            int64
value_regular_seats                  int64
value_kiosk

In [6]:
churn_df.head()

,ws_users_activated,ws_users_deactivated,ws_users_invited,action_create_project,action_export_report,action_api_and_webhooks,action_time_entries_via_tracker,action_start_trial,action_import_csv,action_create_invoice,action_lock_entries,action_add_targets,action_connect_quickbooks,action_create_expense,action_project_budget,action_gps_tracking,action_screenshots,action_create_custom_field,country,value_days_to_purchase,value_number_of_active_months,value_transactions_number,value_regular_seats,value_kiosk_seats,revenue,churned_status
0,3,2,0,5,8,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Canada,2,0,6,3,0,184.925,No
1,6,1,0,35,106,0,33,0,1,0,5,8,0,0,3,NaN,NaN,3.0,United Kingdom,37,9,9,6,0,608.842,No
2,2,0,0,3,3,0,0,0,0,0,10,2,1,0,9,NaN,1.0,NaN,Florida,98,3,12,3,0,395.122,No
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,NaN,Kentucky,0,1,2,1,0,25.974,Yes
4,1,0,0,0,0,1,0,0,0,0,0,0,2,0,0,NaN,NaN,NaN,Ireland,21,2,3,1,0,38.961,Yes


## Null values handling 

In [7]:
missing_values = churn_df.isnull().sum()
print(missing_values)

ws_users_activated                    0
ws_users_deactivated                  0
ws_users_invited                      0
action_create_project                 0
action_export_report                  0
action_api_and_webhooks               0
action_time_entries_via_tracker       0
action_start_trial                    0
action_import_csv                     0
action_create_invoice                 0
action_lock_entries                   0
action_add_targets                    0
action_connect_quickbooks             0
action_create_expense                 0
action_project_budget                 0
action_gps_tracking                1626
action_screenshots                 1458
action_create_custom_field         2059
country                              84
value_days_to_purchase                0
value_number_of_active_months         0
value_transactions_number             0
value_regular_seats                   0
value_kiosk_seats                     0
revenue                               0


As we can see here, a lot of people didn't want their GPS to be tracked, and they didn't add screenshots. So we can discard those columns entirely as they are empty on more thatn 50% of the ocassions. Also, we could exclude records that don't have a country, because they make a really small sample.

In [8]:
columns_to_exclude = ['action_gps_tracking', 'action_screenshots', 'action_create_custom_field']
churn_df = churn_df.drop(columns=columns_to_exclude)

In [9]:
churn_df = churn_df[churn_df['country'].notnull()]

In [10]:
missing_values = churn_df.isnull().sum()
print(missing_values)

ws_users_activated                 0
ws_users_deactivated               0
ws_users_invited                   0
action_create_project              0
action_export_report               0
action_api_and_webhooks            0
action_time_entries_via_tracker    0
action_start_trial                 0
action_import_csv                  0
action_create_invoice              0
action_lock_entries                0
action_add_targets                 0
action_connect_quickbooks          0
action_create_expense              0
action_project_budget              0
country                            0
value_days_to_purchase             0
value_number_of_active_months      0
value_transactions_number          0
value_regular_seats                0
value_kiosk_seats                  0
revenue                            0
churned_status                     0
dtype: int64


In [11]:
churn_df.dtypes

ws_users_activated                   int64
ws_users_deactivated                 int64
ws_users_invited                     int64
action_create_project                int64
action_export_report                 int64
action_api_and_webhooks              int64
action_time_entries_via_tracker      int64
action_start_trial                   int64
action_import_csv                    int64
action_create_invoice                int64
action_lock_entries                  int64
action_add_targets                   int64
action_connect_quickbooks            int64
action_create_expense                int64
action_project_budget                int64
country                             object
value_days_to_purchase               int64
value_number_of_active_months        int64
value_transactions_number            int64
value_regular_seats                  int64
value_kiosk_seats                    int64
revenue                            float64
churned_status                      object
dtype: obje

Now the data doesn't have any missing values. 

Every column is now numeric, except the country and churned status. 

# Outliers handling

In [ ]:
churn_df.describe()

,ws_users_activated,ws_users_deactivated,ws_users_invited,action_create_project,action_export_report,action_api_and_webhooks,action_time_entries_via_tracker,action_start_trial,action_import_csv,action_create_invoice,action_lock_entries,action_add_targets,action_connect_quickbooks,action_create_expense,action_project_budget,action_gps_tracking,action_screenshots,action_create_custom_field,value_days_to_purchase,value_number_of_active_months,value_transactions_number,value_regular_seats,value_kiosk_seats,revenue
count,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,876.000000,1044.000000,443.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000
mean,5.619504,0.827738,0.158273,28.043965,22.709432,0.383293,19.479616,0.175859,0.622702,8.494005,1.634293,0.290568,0.081934,10.019185,10.459233,1.371005,1.417625,7.246050,61.286571,4.215827,5.728617,6.067946,0.257794,378.331825
std,11.364130,3.527056,0.784527,80.761092,80.884964,3.089846,114.856050,0.380777,4.770705,52.699928,7.180274,1.319093,0.688108,72.849346,37.851112,0.726969,0.791806,11.577418,85.179584,3.691711,4.893211,11.766325,2.957970,1007.971191
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,38.961000
50%,2.000000,0.000000,0.000000,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,4.000000,24.000000,3.000000,4.000000,2.000000,0.000000,105.761500
75%,6.000000,0.000000,0.000000,26.000000,15.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2.000000,2.000000,7.000000,84.750000,7.000000,8.000000,6.000000,0.000000,333.459750
max,206.000000,73.000000,20.000000,1923.000000,1740.000000,127.000000,3382.000000,1.000000,120.000000,1405.000000,152.000000,30.000000,27.000000,1785.000000,829.000000,8.000000,11.000000,106.000000,420.000000,14.000000,90.000000,215.000000,117.000000,27235.156000


Looking at this table, we can see that a lot of data has been skewed by the maximum values (probably large clients). Let's take for example `action_api_and_webhooks`. Mean of that column is 0.38 and std is around 3.086. But the maximum value is 127, which skews picture heavily.

Before we jump into solving this issue, we have to see that all of the numbers are non-negative. So we will trim down all of the top 3% for each of the column. This won't be done iteratively, because that will lead to having $(0.97)^{num\_ of\_ columns}$ which would lead to having very small sample of data to train the model on (especially in our case where we have a lot of columns). So we will create a function that tests this for all of the numeric columns.

In [67]:
def calculate_quantiles(dataframe:pd.DataFrame) -> dict:
    """
    Calculate the 97% quantile for each numeric column in a Pandas DataFrame.

    Parameters:
    dataframe (pd.DataFrame): The input DataFrame.

    Returns:
    (dict): A dictionary where keys are numeric column names, and values are the 97% quantiles.
    """
    quantiles_dict = {}
    
    # Select only numeric columns
    numeric_columns = dataframe.select_dtypes(include=[np.number])
    
    # Loop through each numeric column
    for column in numeric_columns.columns:
        # Calculate the 98% quantile for the column
        quantile_97 = numeric_columns[column].quantile(0.97)
        # Add the result to the dictionary
        quantiles_dict[column] = quantile_97
    
    return quantiles_dict

In [68]:
quantile_97th = calculate_quantiles(churn_df)

In [69]:
def is_record_within_quantiles(record: pd.Series) -> bool:
    """
    Check if a single record (row) falls within the quantile values specified in a dictionary.

    Parameters:
    record (pd.Series): A single row (record) from a DataFrame.
    quantile_dict (dict): A dictionary where keys are column names, and values are the quantile values.

    Returns:
    (bool): True if the record falls within the quantiles for all columns, False otherwise.
    """
    # Initialize a variable to track whether the record is within quantiles
    is_within_quantiles = True

    # Loop through each column in the record
    for column in quantile_97th.keys():
        value = record[column]
        # Check if the record value is less than or equal to the quantile value for the column
        if value > quantile_97th[column]:
            is_within_quantiles = False
            break  # Exit the loop early if a column is not within quantiles

    return is_within_quantiles

In [70]:
churn_df['isWithinQuantile'] = churn_df.apply(lambda row: is_record_within_quantiles(row), axis=1)


In [71]:
churn_df['isWithinQuantile'].value_counts()

isWithinQuantile
True     1761
False     657
Name: count, dtype: int64

In [72]:
no_outlier_churf_df = churn_df[churn_df['isWithinQuantile'] == True]
no_outlier_churf_df.drop(['isWithinQuantile'], inplace=True, axis=1)

C:\Users\DT User\AppData\Local\Temp\ipykernel_5048\2419240485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_outlier_churf_df.drop(['isWithinQuantile'], inplace=True, axis=1)


In [73]:
no_outlier_churf_df.describe()

,ws_users_activated,ws_users_deactivated,ws_users_invited,action_create_project,action_export_report,action_api_and_webhooks,action_time_entries_via_tracker,action_start_trial,action_import_csv,action_create_invoice,action_lock_entries,action_add_targets,action_connect_quickbooks,action_create_expense,action_project_budget,value_days_to_purchase,value_number_of_active_months,value_transactions_number,value_regular_seats,value_kiosk_seats,revenue
count,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000
mean,2.886428,0.246451,0.060761,13.088586,7.477002,0.103918,3.677456,0.147076,0.063032,2.829074,0.542873,0.087450,0.027825,1.609313,3.725724,40.718342,3.558206,4.498580,3.199886,0.018739,157.688620
std,3.159697,0.747359,0.238959,20.511751,17.029442,0.340440,14.902921,0.354282,0.376892,7.388703,1.724986,0.340898,0.164518,6.526311,10.463642,60.644738,3.239343,3.340033,3.418835,0.135642,222.013623
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,0.000000,25.974000
50%,2.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.000000,3.000000,4.000000,2.000000,0.000000,77.922000
75%,4.000000,0.000000,0.000000,16.000000,6.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,51.000000,6.000000,6.000000,4.000000,0.000000,181.779000
max,24.000000,5.000000,1.000000,147.000000,129.000000,2.000000,138.000000,1.000000,4.000000,51.000000,13.000000,2.000000,1.000000,65.000000,82.000000,285.000000,12.000000,16.000000,24.000000,1.000000,1793.857000


In [74]:
no_outlier_churf_df['churned_status'].value_counts()

churned_status
No     1110
Yes     651
Name: count, dtype: int64

In [76]:
no_outlier_churf_df.to_csv("../data/task_data_churned_clean.csv")

As we can see here, results look mode natural. Of course, we wouldn't exlcude a lot of those, because we would lose a general pattern. But those ones that heavily were skewing the picture are now removed.

Another way at tackling this would be taking interval $(mean - 3*std, mean + 3*std)$ which is also pretty common. But due to lack of time and testing capabilties we have chosen approach shown above.